The following notebook was used as the starting point: https://github.com/SheffieldML/notebook/blob/master/GPy/sparse_gp_regression.ipynb

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import GPy
import numpy as np
import matplotlib.pyplot as plt
from GPy.core.parameterization.variational import NormalPosterior
np.random.seed(101)

In [ ]:
def plot_covariance_matrix(cov_matrix):
    fig, ax = plt.subplots(figsize=(8,8))
    im = ax.imshow(cov_matrix, interpolation='none')
    fig.colorbar(im)

## Sample Function

Now we'll sample a Gaussian process regression problem directly from a Gaussian process prior. We'll use an exponentiated quadratic covariance function with a lengthscale and variance of 1 and sample 50 equally spaced points. 

In [ ]:
N = 50
noise_var = 0.05

X = np.linspace(0,10,50)[:,None]
X_new = np.linspace(10,15,50)[:,None]
k = GPy.kern.RBF(1)
y = np.random.multivariate_normal(np.zeros(N),k.K(X)+np.eye(N)*np.sqrt(noise_var)).reshape(-1,1)

## Full Gaussian Process Fit

Now we use GPy to optimize the parameters of a Gaussian process given the sampled data. Here, there are no approximations, we simply fit the full Gaussian process.

In [ ]:
m_full = GPy.models.GPRegression(X,y)
m_full.optimize('bfgs')
m_full.plot()
print(m_full)

## Playground

In [ ]:
# Z = np.hstack((np.linspace(2.5,4.,3),np.linspace(7,8.5,3)))[:,None]
# m = GPy.models.SparseGPRegression(X,y,Z=Z)
m = GPy.models.SparseGPRegression(X,y, num_inducing=6)
m.likelihood.variance = noise_var
# m.inducing_inputs.fix()
m.rbf.variance.fix()
m.rbf.lengthscale.fix()
m.Z.unconstrain()
m.optimize('bfgs')
# m.optimize('bfgs', messages=True)
m.plot()
print(m)

In [ ]:
m.plot_f()

In [ ]:
m.plot_density()

### KL divergence

In [ ]:
# samples = X = np.linspace(0,10,1000)[:,None]
samples = X
# samples = m.Z

In [ ]:
mu, covar = m.predict_noiseless(samples, full_cov=True)
variances = covar.diagonal()
variances = np.reshape(variances, (len(samples), 1))
sparse_posterior = NormalPosterior(means=mu, variances=variances)

In [ ]:
mu_full, covar_full = m_full.predict_noiseless(samples, full_cov=True)
variances_full = covar_full.diagonal()
variances_full = np.reshape(variances, (len(samples), 1))
full_posterior = NormalPosterior(means=mu_full, variances=variances_full)

In [ ]:
divergence = sparse_posterior.KL(full_posterior)
print(divergence)

In [ ]:
log_likelihood_sparse_model = m.log_likelihood()[0][0]
log_likelihood_full_model = m_full.log_likelihood()
likelihood_sparse_model = np.e**log_likelihood_sparse_model
likelihood_full_model = np.e**log_likelihood_full_model
print(log_likelihood_sparse_model, log_likelihood_full_model)
print(likelihood_sparse_model, likelihood_full_model)
print(log_likelihood_sparse_model - log_likelihood_full_model)
print(likelihood_sparse_model - likelihood_full_model)

## !!!!!!!!!
^^ This time, we have enough inducing points and the fit resembles that of the GP. This is verified by the fact that the bound on the marginal likelihood is tight, which means that our variational approximation must be good (the difference between the bound and the true likelihood is the Kullback Leibler divergence between the approximation and the truth). 

### Show covar of inducing inputs and of full gp

In [ ]:
plot_covariance_matrix(covar)
plot_covariance_matrix(covar_full)

## Check posterior over inducing points

In [ ]:
posterior_covariance = m.posterior.covariance
posterior_mean = m.posterior.mean

In [ ]:
plot_covariance_matrix(posterior_covariance)